In this notebook, we are deriving what the alpha value in the @dev formula is based off of 1hive Sol [code](https://github.com/1Hive/conviction-voting-app/blob/bc81f4af266bab6bc4096a31966bd9fe8a89f4a2/contracts/ConvictionVoting.sol)

    /**
     * @dev Formula: ρ * supply / (1 - a) / (β - requestedAmount / total)**2
     * For the Solidity implementation we amplify ρ and β and simplify the formula:
     * weight = ρ * D
     * maxRatio = β * D
     * decay = a * D
     * threshold = weight * supply * D ** 2 * funds ** 2 / (D - decay) / (maxRatio * funds - requestedAmount * D) ** 2
     * @param _requestedAmount Requested amount of tokens on certain proposal
     * @return Threshold a proposal's conviction should surpass in order to be able to
     * executed it.
     */
    function calculateThreshold(uint256 _requestedAmount) public view returns (uint256 _threshold) {
        uint256 funds = vault.balance(requestToken);
        require(maxRatio.mul(funds) > _requestedAmount.mul(D), ERROR_AMOUNT_OVER_MAX_RATIO);
        uint256 supply = stakeToken.totalSupply();
        // denom = maxRatio * 2 ** 64 / D  - requestedAmount * 2 ** 64 / funds
        uint256 denom = (maxRatio << 64).div(D).sub((_requestedAmount << 64).div(funds));
        // _threshold = (weight * 2 ** 128 / D) / (denom ** 2 / 2 ** 64) * supply * D / 2 ** 128
        _threshold = ((weight << 128).div(D).div(denom.mul(denom) >> 64)).mul(D).div(D.sub(decay)).mul(supply) >> 64;
    }

In [1]:
import numpy as np
# Z original code without alpha
def trigger_threshold_original(requested, funds, supply, beta, rho):
    '''
    Function that determines threshold for proposals being accepted. 
    '''
    share = requested/funds
    if share < beta:
        return rho*supply/(beta-share)**2
    else: 
        return np.inf
    
#Our new code code
def trigger_threshold(requested, funds, supply, beta, rho, alpha):
    '''
    Function that determines threshold for proposals being accepted. 
    '''
    share = requested/funds
    if share < beta:
        threshold = rho*supply/(beta-share)**2  * 1/(1-alpha)
        return threshold #* (1-alpha)
    else: 
        return np.inf
    
# Assumed values
funds = 4000
requested = 100
supply = 21706
share = requested/funds
share

0.025

In [2]:
# Sol values
D = 10000000
decay = 9999599
maxRatio = 2000000
weight = 20000

Based on the above equations, we can see that $\rho$ is equal to:

In [3]:
rho = weight / D
rho

0.002

We can derive $\beta$ is equal to be:

In [4]:
beta = maxRatio/D
beta

0.2

In [5]:
originalOutput = trigger_threshold_original(requested,funds,supply,beta,rho)
originalOutput

1417.5346938775506

sol code threshold:

In [31]:
solThreshold = weight * supply * D ** 2 * funds ** 2 / (D - decay) / (maxRatio * funds - requested * D) ** 2
solThreshold

35349992.36602372

Our new code threshold to match 1hive dev formulata 

In [15]:
alpha = 1- 0.9999599
"{:.7f}".format(alpha)

'0.0000401'

In [19]:
newThreshold = trigger_threshold(requested, funds, supply, beta, rho, alpha)
newThreshold

1417.5915392982765